In [1]:
#project: p12
#submitter: bodla
#partner: kksun

## Self-Reminder: if low score check the haversine function

In [2]:
import os, requests, bs4, json
import pandas as pd, numpy as np
from math import radians, cos, sin, asin, sqrt

In [3]:
#lab-12 Functions

def download(filename, url):
    # We do not download again if the file already exists
    if os.path.exists(filename):
        return (str(filename) + " already exists!")

    # TODO: Write the code to download the file from URL
    # and save it in `filename`
    
    res = requests.get(url)
    res.raise_for_status()
    
    with open(filename, "w") as file:
        file.write(res.text)
    
    return (str(filename) + " created!")

In [4]:
#setup

download("countries.json", "https://raw.githubusercontent.com/msyamkumar/cs220-f20-projects/master/p12/countries.json")
countries = pd.read_json("countries.json")
# countries.head()

download("capitals.json", " https://raw.githubusercontent.com/msyamkumar/cs220-f20-projects/master/p12/capitals.json")
capitals = pd.read_json("capitals.json")

In [5]:
countries.head()
capitals.head()

,country,capital,latitude,longitude
0,Afghanistan,Kabul,34.516667,69.183333
1,Albania,Tirana,41.316667,19.816667
2,Algeria,Algiers,36.750000,3.050000
3,Anguilla,The Valley,18.216667,-63.050000
4,Argentina,Buenos Aires,-34.583333,-58.666667


In [6]:
#q1
len(countries)

174

In [7]:
#q2
countries['population'].sum()

6261901793

In [8]:
#q3
caps = list(capitals['capital'])
caps.sort()
caps

['Abu Dhabi',
 'Abuja',
 'Accra',
 'Addis Ababa',
 'Algiers',
 'Amman',
 'Amsterdam',
 'Ankara',
 'Antananarivo',
 'Apia',
 'Ashgabat',
 'Asmara',
 'Astana',
 'Asuncion',
 'Athens',
 'Baghdad',
 'Baku',
 'Bamako',
 'Bangkok',
 'Beijing',
 'Beirut',
 'Belmopan',
 'Berlin',
 'Bern',
 'Bishkek',
 'Bissau',
 'Bogota',
 'Brasilia',
 'Bridgetown',
 'Brussels',
 'Bucharest',
 'Budapest',
 'Buenos Aires',
 'Bujumbura',
 'Cairo',
 'Canberra',
 'Caracas',
 'Castries',
 'Chisinau',
 'Colombo',
 'Conakry',
 'Copenhagen',
 'Dakar',
 'Damascus',
 'Dar es Salaam',
 'Dhaka',
 'Djibouti',
 'Doha',
 'Dublin',
 'Dushanbe',
 'Freetown',
 'Gaborone',
 'George Town',
 'Georgetown',
 'Guatemala City',
 'Hagatna',
 'Hamilton',
 'Hanoi',
 'Harare',
 'Havana',
 'Helsinki',
 'Islamabad',
 'Jakarta',
 'Jamestown',
 'Jerusalem',
 'Kabul',
 'Kampala',
 'Kathmandu',
 'Khartoum',
 'Kigali',
 'Kingston',
 'Kingstown',
 'Kuala Lumpur',
 'Kuwait City',
 'Kyiv',
 'La Paz',
 'Libreville',
 'Lilongwe',
 'Lima',
 'Lisbon',


In [9]:
#q4
next(iter((capitals[capitals['country'] == "Italy"])['capital']))

'Rome'

In [10]:
#q5
next(iter((capitals[capitals['capital'] == "Brussels"])['country']))

'Belgium'

In [11]:
#q6
list(capitals.sort_values(by = 'latitude').head(7)['country'])

['New Zealand',
 'Australia',
 'Uruguay',
 'Argentina',
 'Chile',
 'Lesotho',
 'Swaziland']

In [12]:
#q7
list(capitals.sort_values(by = 'latitude', ascending = False).head(10)['country'])

['Iceland',
 'Finland',
 'Norway',
 'Estonia',
 'Sweden',
 'Latvia',
 'Russia',
 'Denmark',
 'Lithuania',
 'Belarus']

In [13]:
#q8
next(iter(countries[(countries['continent'] == 'Asia') & (countries['coastline'] == 0)].sort_values(by = 'area')['country']))

'Bhutan'

In [14]:
#q9
next(iter(countries[(countries['continent'] == 'Asia') & (countries['coastline'] != 0)].sort_values(by = 'area', ascending = False)['country']))

'China'

In [15]:
#q10
next(iter(countries[(countries['continent'] == 'South America') & (countries['coastline'] != 0)].sort_values(by = 'population')['country']))

'Suriname'

In [16]:
#q11

#code from: https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
def haversine(lat1, lon1, lat2, lon2):
    R = 3959.87433 # this is in miles.  For Earth radius in kilometers use 6372.8 km

    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return R * c

rand = (43.070231, -89.411893)
wcap = (43.074645, -89.384113)

haversine(rand[0], rand[1], wcap[0], wcap[1])

# print(haversine(lat1, lon1, lat2, lon2))

1.4353037892976752

In [17]:
#q12

def get_country_dist(country1, country2, df = capitals):
    #find each index for the capital, then extract capital coordinates
    indx1 = df[(df['country'] == country1)].index[0]
    indx2 = df[(df['country'] == country2)].index[0]
    
    coord1 = (df.loc[indx1, 'latitude'], df.loc[indx1, 'longitude'])
    coord2 = (df.loc[indx2, 'latitude'], df.loc[indx2, 'longitude'])
    
    #caluclate & return distances
    return haversine(coord1[0], coord1[1], coord2[0], coord2[1])

get_country_dist('Germany', 'Norway')

521.3682875349596

In [18]:
#q13

#returns a dataframe with only the indicated countries from capitals
def many_country_dist(cntr_lis):
    #create a dictionary of dictionaries for the distances
    distances = {}
    
    for country in cntr_lis:
        distances[country] = {}
        
        for ctry in cntr_lis:
            if country == ctry:
                distances[country][ctry] = np.nan
                continue
            
            distances[country][ctry] = get_country_dist(country, ctry)
            
    
    return pd.DataFrame(distances)

many_country_dist(['Italy', 'United Kingdom', 'Germany'])

,Italy,United Kingdom,Germany
Italy,NaN,889.162839,734.992885
United Kingdom,889.162839,NaN,577.036561
Germany,734.992885,577.036561,NaN


In [19]:
#q14

#find all countries in North America

na_lis = list(countries[countries['continent'] == 'North America']['country'])
na_dist = many_country_dist(na_lis)
na_dist

,Belize,Jamaica,Bermuda,Saint Vincent and the Grenadines,Cuba,Saint Lucia,United States,Anguilla,Grenada,Dominican Republic,...,Guatemala,Panama,El Salvador,Mexico,Honduras,Aruba,Puerto Rico,Canada,Cayman Islands,Haiti
Belize,NaN,789.791144,1821.403743,1857.709707,580.880630,1860.078929,1654.860542,1692.850059,1840.086574,1243.386123,...,215.976434,844.507030,247.041838,696.467050,240.894903,1292.555970,1491.391224,2087.633669,504.770184,1084.080791
Jamaica,789.791144,NaN,1238.738632,1090.074320,503.890291,1084.971586,1443.356660,903.146658,1084.805921,454.053098,...,939.093571,650.721612,876.005573,1464.231609,742.290237,589.043607,701.921887,1895.901593,313.277087,295.447510
Bermuda,1821.403743,1238.738632,NaN,1342.627755,1245.141911,1285.987010,823.194890,978.156006,1411.587065,1006.526204,...,2027.934603,1867.744359,2006.056866,2300.647782,1891.993102,1406.109694,958.502602,1079.009148,1364.922520,1059.892671
Saint Vincent and the Grenadines,1857.709707,1090.074320,1342.627755,NaN,1547.356439,61.641201,2024.619911,371.885480,83.065697,684.818368,...,1967.264034,1274.893860,1880.594747,2547.527503,1746.833474,595.637766,491.884940,2387.418769,1403.250825,827.649716
Cuba,580.880630,503.890291,1245.141911,1547.356439,NaN,1531.619578,1134.242918,1291.930677,1554.891952,865.760580,...,793.479864,995.569528,790.383476,1109.702542,699.758286,1091.225285,1096.178288,1585.928556,271.030406,720.114524
Saint Lucia,1860.078929,1084.971586,1285.987010,61.641201,1531.619578,NaN,1977.253392,321.631343,143.920226,666.188627,...,1975.779572,1301.912716,1891.308249,2546.729060,1756.795431,616.213270,458.801357,2335.152974,1397.346452,814.270533
United States,1654.860542,1443.356660,823.194890,2024.619911,1134.242918,1977.253392,NaN,1655.822264,2077.409249,1473.981088,...,1868.026258,2073.584680,1893.213449,1884.874493,1822.646663,1871.681009,1554.822257,456.405827,1378.623148,1434.048239
Anguilla,1692.850059,903.146658,978.156006,371.885480,1291.930677,321.631343,1655.822264,NaN,434.921022,449.676684,...,1836.043310,1277.190817,1763.768867,2357.654538,1627.932002,609.536747,201.916585,2015.583485,1201.543175,609.210712
Grenada,1840.086574,1084.805921,1411.587065,83.065697,1554.891952,143.920226,2077.409249,434.921022,NaN,701.110224,...,1941.379761,1226.769438,1851.992731,2533.161919,1719.447961,560.284324,530.404391,2448.006003,1397.794225,835.359466
Dominican Republic,1243.386123,454.053098,1006.526204,684.818368,865.760580,666.188627,1473.981088,449.676684,701.110224,NaN,...,1390.667595,921.004293,1322.427761,1909.798817,1187.102427,411.317194,248.008441,1892.155011,752.996181,159.549243


In [20]:
#q15
na_dist.mean(0, skipna = True).sort_values().index[0]

'Jamaica'

In [21]:
#q16
na_dist.mean(1, skipna = True).sort_values().index[-1]

'Canada'

In [22]:
#q17

neighbors = pd.DataFrame(index=na_dist.index)
neighbors['nearest'] = list(na_dist.idxmin().values) 
neighbors['distance'] =  na_dist.min()

neighbors

,nearest,distance
Belize,Guatemala,215.976434
Jamaica,Haiti,295.447510
Bermuda,United States,823.194890
Saint Vincent and the Grenadines,Saint Lucia,61.641201
Cuba,Cayman Islands,271.030406
Saint Lucia,Saint Vincent and the Grenadines,61.641201
United States,Canada,456.405827
Anguilla,Puerto Rico,201.916585
Grenada,Saint Vincent and the Grenadines,83.065697
Dominican Republic,Haiti,159.549243


In [23]:
#q18

furthest = pd.DataFrame(index=na_dist.index)
furthest['furthest'] = list(na_dist.idxmax().values) 
furthest['distance'] =  na_dist.max()

furthest

,furthest,distance
Belize,Canada,2087.633669
Jamaica,Canada,1895.901593
Bermuda,Mexico,2300.647782
Saint Vincent and the Grenadines,Mexico,2547.527503
Cuba,Barbados,1642.843403
Saint Lucia,Mexico,2546.729060
United States,Grenada,2077.409249
Anguilla,Mexico,2357.654538
Grenada,Mexico,2533.161919
Dominican Republic,Mexico,1909.798817


In [24]:
#q19
q19 = pd.DataFrame()
q19['birth-rate'] = countries['birth-rate'].describe()
q19['death-rate'] = countries['death-rate'].describe()

q19

,birth-rate,death-rate
count,174.000000,174.000000
mean,22.463851,9.625172
std,11.278992,5.187143
min,8.250000,2.410000
25%,12.597500,6.027500
50%,20.010000,8.230000
75%,29.860000,11.715000
max,50.730000,29.740000


In [25]:
capitals.head()

,country,capital,latitude,longitude
0,Afghanistan,Kabul,34.516667,69.183333
1,Albania,Tirana,41.316667,19.816667
2,Algeria,Algiers,36.750000,3.050000
3,Anguilla,The Valley,18.216667,-63.050000
4,Argentina,Buenos Aires,-34.583333,-58.666667


In [26]:
#q20

#read in html and write to file
download('capitals.html', 'https://raw.githubusercontent.com/msyamkumar/cs220-f20-projects/master/p12/techslides-snapshot.html')
with open('capitals.html', 'r') as file:
    caps = bs4.BeautifulSoup(file.read(), 'html.parser')

table = caps.find("table")
orig_cntr = list(countries['country'])

data = []

for tr in table.find_all('tr'):
    
    #first text for each td is the country
    #check if current cuntry is in the original countries list
    if tr.find('td').text not in orig_cntr:
        continue

    #create the data for each country
    curr_data = {'country': None, 'capital': None, 'latitude': None, 'longitude': None}
    for i, td in enumerate(tr.find_all('td')):
        if i == 0:
            curr_data['country'] = td.text
        elif i == 1:
            curr_data['capital'] = td.text
        elif i == 2:
            curr_data['latitude'] = float(td.text)
        elif i == 3:
            curr_data['longitude'] = float(td.text)
    data.append(curr_data)

with open("my_capitals.json", 'w') as file:
    json.dump(data, file)

with open("my_capitals.json") as file:
    jsn = (file.read())
    
jsn

'[{"country": "Afghanistan", "capital": "Kabul", "latitude": 34.51666667, "longitude": 69.183333}, {"country": "Albania", "capital": "Tirana", "latitude": 41.31666667, "longitude": 19.816667}, {"country": "Algeria", "capital": "Algiers", "latitude": 36.75, "longitude": 3.05}, {"country": "Anguilla", "capital": "The Valley", "latitude": 18.21666667, "longitude": -63.05}, {"country": "Argentina", "capital": "Buenos Aires", "latitude": -34.58333333, "longitude": -58.666667}, {"country": "Armenia", "capital": "Yerevan", "latitude": 40.16666667, "longitude": 44.5}, {"country": "Aruba", "capital": "Oranjestad", "latitude": 12.51666667, "longitude": -70.033333}, {"country": "Australia", "capital": "Canberra", "latitude": -35.26666667, "longitude": 149.133333}, {"country": "Austria", "capital": "Vienna", "latitude": 48.2, "longitude": 16.366667}, {"country": "Azerbaijan", "capital": "Baku", "latitude": 40.38333333, "longitude": 49.866667}, {"country": "Bahrain", "capital": "Manama", "latitude"